In [ ]:
import pandas as pd
import numpy as np
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from getpass import getpass
import time

In [ ]:
# Input LinkedIn Login Credentials
username = input("Enter your email/phone number: ")
password = getpass("Enter your password: ")

In [ ]:
# Initiate Chrome Driver
# Ensure that your computer has chrome driver downloaded and located in the same directory as this script
# If scaling, we can containerize all of the necessary dependencies to ensure script runs seamlessly (or run and structed a cloud based job)
driver = webdriver.Chrome()
driver.get("https://www.linkedin.com/login?fromSignIn=true&trk=guest_homepage-basic_nav-header-signin")

# Input LinkedIn Username
username_input = driver.find_element(By.ID, "username")
username_input.send_keys(username)

# Input LinkedIn Password
password_input = driver.find_element(By.ID, "password")
password_input.send_keys(password)

# Press 'enter' key after credentials are inserted
actions = ActionChains(driver)
actions.send_keys(Keys.ENTER)
actions.perform()

# Set delay to give user time to confirm login via MFA
# Can be improved and scaled using Try/Except error handling clauses to ensure script does not break
# Click on the 'Jobs' tab on the LinkedIn home page once logged in
time.sleep(45)
job_button = driver.find_element(By.CSS_SELECTOR, "[href='https://www.linkedin.com/jobs/?']")
job_button.click()

# Click on the 'My Jobs' sub-tab on the LinkedIn 'Jobs' page
time.sleep(5)
my_jobs = driver.find_element(By.PARTIAL_LINK_TEXT, "My jobs")
my_jobs.click()

# Click on 'Applied' button/bubble 
time.sleep(5)
applications_xpath = '//*[@id="search-reusables__filters-bar"]/ul/li[3]/button'
applied_button = driver.find_element(By.XPATH, applications_xpath)
applied_button.click()

In [ ]:
# Set the amount of pages we want to scrape job data from (page_amount)
time.sleep(5)
page_amount = 2
universal_xpath = f"/html/body/div[5]/div[3]/div/main/section/div/div[2]/div/ul"

job_titles = []
companies = []
locations = []
application_time = []

for i in range(page_amount):
    
    # Determine the maximum number of jobs being displayed in the jobs page
    page_max = int(len(driver.find_element(By.XPATH, universal_xpath).text.split("\n"))/4)
    job_listing = list(range(1,page_max+1))

    # Scrape and parse job data and populate the 4 empty lists above (job_titles, companies, locations, application_time)
    for x in job_listing:
        XPATH = f"/html/body/div[5]/div[3]/div/main/section/div/div[2]/div/ul/li[{x}]"
        job = driver.find_element(By.XPATH, XPATH)
        job_data = str(job.text).split("\n")
        job_titles.append(job_data[0])
        companies.append(job_data[1])
        locations.append(job_data[2])
        application_time.append(job_data[3])
        print(job_data)

    # Click the 'Next' button on the bottom of webpage to automatically move to the next page
    next_page_button = driver.find_element(By.XPATH, '/html/body/div[5]/div[3]/div/main/section/div/div[3]/button[2]')

    next_page_button.text
    next_page_button.click()
    time.sleep(5)

In [ ]:
# Combine data and structure as pandas dataframe
data = list(zip(job_titles, companies, locations, application_time))
column_names = ['job_title', 'company', 'location', 'application_time']

df = pd.DataFrame(data, columns= column_names)

# Export CSV file containing this data
file_name = f"{username} LinkedIn Job Applications.csv"
df.to_csv(file_name, index = False)